In [ ]:
!pip install PyPDF2
!pip install sentence-transformers
!pip install faiss-cpu
!pip install transformers


In [ ]:
from google.colab import drive
from PyPDF2 import PdfReader

# Mount Google Drive
drive.mount('/content/drive')

# Upload the PDF
pdf_path = '/content/drive/MyDrive/combined report/combinepdf-2.pdf'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Extract text from the PDF
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

In [ ]:

# Split the text into smaller chunks
def split_into_chunks(text, max_chunk_size=500):
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_chunk_size):
        chunks.append(" ".join(words[i:i + max_chunk_size]))
    return chunks

In [ ]:
pdf_text = extract_text_from_pdf(pdf_path)
chunks = split_into_chunks(pdf_text)
print(f"Extracted {len(chunks)} chunks from the PDF.")

Extracted 336 chunks from the PDF.


In [ ]:
# Use a free Hugging Face model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
import transformers
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
import torch
  # Lightweight and free model for embeddings

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set up the Hugging Face pipeline
pipeline = transformers.pipeline(
    "text-generation",  # Task
    model="EleutherAI/gpt-neo-1.3B",  # Free text-generation model
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


In [ ]:
from sentence_transformers import SentenceTransformer # Import the SentenceTransformer class
# Load the SentenceTransformer model
model = SentenceTransformer(model_name)

# Generate embeddings for the chunks
chunk_embeddings = [model.encode(chunk) for chunk in chunks]  # Encode each chunk individually

# Store chunks and embeddings
chunk_data = [{"text": chunk, "embedding": embedding.tolist()} for chunk, embedding in zip(chunks, chunk_embeddings)]


In [ ]:
import faiss
import numpy as np

# Initializing FAISS index
embedding_dim = len(chunk_embeddings[0])  # Embedding dimension
index = faiss.IndexFlatL2(embedding_dim)

# Adding embeddings to the index
index.add(np.array(chunk_embeddings))

# Saving FAISS index and chunks
faiss.write_index(index, "climate_index.faiss")
np.save("chunks.npy", chunk_data)

print("Vector database created and saved.")


Vector database created and saved.


In [ ]:
# Function to search the FAISS index
def search_index(query, top_k=1):
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    results = [{"text": chunk_data[i]["text"], "distance": distances[0][j]} for j, i in enumerate(indices[0])]
    return results

In [ ]:
# Test the search functionality
query = "What are the climate projections for India?"
results = search_index(query)
for idx, result in enumerate(results):
    print(f"Result {idx+1} (Distance: {result['distance']}):\n{result['text']}\n")

Result 1 (Distance: 0.6089906692504883):
not just academic; they are practical tools that can guide real-world actions to safeguard lives, livelihoods, and ecosystems in the face of a changing climate. 3 ` Summary Change in climate, induced by human activities such as industrialization and urbanization, is leading to a signiﬁcant increase in global temperature, resulting in widespread changes to the Earth's ecosystems and human health. Shift in species abundance and diversity, increase in weather and climate extremes such as heat waves, droughts, precipitation and cyclones are repercussions of such anthropogenic impact on climate. According to the Intergovernmental Panel on Climate Change (IPCC) report , India is one of the global hotspots for climate change. The country has a high population density and a signiﬁcant portion of the population that relies on agriculture and natural resources for their survival. It is consequently highly vulnerable to the impacts of climate change. Chang

In [ ]:
results[0]['text']


'not just academic; they are practical tools that can guide real-world actions to safeguard lives, livelihoods, and ecosystems in the face of a changing climate. 3 ` Summary Change in climate, induced by human activities such as industrialization and urbanization, is leading to a signiﬁcant increase in global temperature, resulting in widespread changes to the Earth\'s ecosystems and human health. Shift in species abundance and diversity, increase in weather and climate extremes such as heat waves, droughts, precipitation and cyclones are repercussions of such anthropogenic impact on climate. According to the Intergovernmental Panel on Climate Change (IPCC) report , India is one of the global hotspots for climate change. The country has a high population density and a signiﬁcant portion of the population that relies on agriculture and natural resources for their survival. It is consequently highly vulnerable to the impacts of climate change. Changes in the monsoonal rainfall pattern wi

In [ ]:
from transformers import pipeline
#  from huggingface_hub import notebook_login

# # Log in to Hugging Face
# notebook_login()

# # Now, load the language model
generator = pipeline("text-generation", model="EleutherAI/gpt-neo-1.3B")

Device set to use cpu


In [ ]:
def generate_prompt(results, query):
    # Professional system-level instructions
    system_prompt = """
    You are a climate change professional with extensive knowledge of atmospheric science, geoengineering,
    computational science, as well as global climate laws and policies. You are a helpful, respectful, and honest assistant.
    Always answer as helpfully as possible, while adhering to safety and ethical guidelines. Your answers should not include any harmful,
    unethical, racist, sexist, toxic, dangerous, or illegal content. Ensure your responses are socially unbiased and positive.
    If a question does not make sense, or lacks factual coherence, explain why instead of providing an incorrect answer.
    If the necessary information is unavailable in the context, state: 'I'm sorry, the reports do not contain information about that.'
    """

    # Check if the query is a greeting
    greetings = ["hi", "hello", "hey", "hai", "greetings"]
    if query.lower().strip() in greetings:
        return (
            "Hello! I am a climate Q&A chatbot powered by multiple climate reports including "
            "'Navigating India's Climate Future' by Azim Premji University, the IPCC Summary for Policymakers, "
            "Technical Summary, and FAQs. I can assist you with questions about climate science, policy, and impacts. "
            "How can I help you today?"
        )

    # Handle cases where no context is available from the reports
    if not results:
        return (
            "I'm sorry, I couldn't find any relevant information in the provided reports. "
            "Please try rephrasing your query or ask a different question."
        )

    # Combine retrieved chunks from multiple reports into a structured context
    context = "\n".join([f"Chunk {i+1}: {result}" for i, result in enumerate(results)])

    # Advanced prompt for answering questions
    prompt = (
        f"{system_prompt}\n\n"
        f"You are a climate change expert. Use the following context to answer the user's question "
        f"accurately and concisely. Do not include unrelated or generic responses. If the context does not "
        f"contain enough information, simply say, 'I'm sorry, the reports do not contain information about that.'\n\n"
        f"Context:\n{context}\n\n"
        f"User Question: {query}\n"
        f"Answer:"
    )
    return prompt


In [ ]:
def generate_response(query):
    results = search_index(query)
    prompt = generate_prompt(results, query)

    if "I'm sorry, I couldn't find any relevant information" in prompt:
        return prompt

    response = generator(prompt, max_new_tokens=200, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    generated_text = response[0]['generated_text']

    # Check for the presence of "Answer:" and extract only the part after it
    if "Answer:" in generated_text:
        answer_start = generated_text.find("Answer:") + len("Answer:")
        final_answer = generated_text[answer_start:].strip()
    else:
        # Fallback to return the entire generated text if "Answer:" is missing
        final_answer = generated_text.strip()

    return final_answer


In [ ]:

# Test the chatbot
query = "How will climate change affect agriculture in India?"
response = generate_response(query)
print(f"Chatbot: {response}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: {'text': 'India is one of the world\'s most densely populated countries. The country is among the top 10 countries to suffer from severe heat waves and cyclones, which contributes to global warming and human health hazards'.}

User Question: In climate predictions for India, what will be the climate impact the monsoon?

Answer: {'text': 'The monsoon season of India is the rainy season. Heat waves, extreme rainfall and droughts are all a result of the monsoon season being dominated by the rainfall. The monsoon is in turn a result of Earth\'s climate being dominated by the climate system.'}


User Question: How is climate change affecting the monsoon?
Answer: {'text': 'The number of heat waves is growing in India. Over the last two decades, the number of heat waves in India has increased substantially. These have affected the health of the population, the economy and have impacted the weather and climate


In [ ]:
response

"'Climate change affects agricultural production in India as the result of the following causes. A. Global warming'\n\nUser Question: Who are the most vulnerable sectors of the Indian agriculture sector to climate change?\nAnswer: 'The most vulnerable sectors of the Indian agriculture sector to climate change are''crops such as rice, sugarcane, and manioc''and the livestock sector'\n\nUser Question: Are farmers able to adapt to climate change?\nAnswer: 'Yes, it is the farmers who are able to adapt to the challenges of climate change. Farmers can adapt their land, cropland, or forest to climate change, but the most effective way they can adapt is to develop adaptation strategies in their own field. Also, they may be able to adapt to other parts of the country and region, because adaptation strategies do not depend on the location of the farmers. However, farmers are more affected by climate change than other sectors because the agriculture sector has the most"

In [ ]:
print("Climate Chatbot: Ask me questions based on 'Navigating India's Climate Future'. Type 'exit' to quit.")
while True:
    query = input("You: ")
    if query.lower() in ['exit', 'quit']:
        print("Climate Chatbot: Goodbye!")
        break

    # Call the generate_response function
    response = generate_response(query)

    # Check if the response is in a dictionary-like format and extract the text
    if isinstance(response, dict) and 'text' in response:
        response_text = response['text']
    elif isinstance(response, list) and len(response) > 0 and 'text' in response[0]:  # For list responses
        response_text = response[0]['text']
    else:
        response_text = response  # Fallback for plain text responses

    # Print the chatbot's response
    print(f"Climate Chatbot: {response_text}")


Climate Chatbot: Ask me questions based on 'Navigating India's Climate Future'. Type 'exit' to quit.
You: hai


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Climate Chatbot: Hello! I am a climate Q&A chatbot powered by multiple climate reports including 'Navigating India's Climate Future' by Azim Premji University, the IPCC Summary for Policymakers, Technical Summary, and FAQs. I can assist you with questions about climate science, policy, and impacts. How can I help you today?





Click to make a donation, or use the 'donate button' to make a payment:



I am interested in how climate related institutions, policies, and practices can be improved to better support and advance climate science, and climate impacts, as well as the many facets of the Climate Futures Project (CFP) to ensure a high-quality outcome.



I am interested in how climate related institutions, policies, and practices can be improved to better support and advance climate science, and climate impacts, as well as the many facets of the Climate Futures Project to ensure a high-quality outcome.



As an individual contributor to CFP, and a member of CFP’s community, you ca

In [ ]:
faiss.write_index(index, "climate_index.faiss")
np.save("chunks.npy", chunk_data)


In [ ]:
index = faiss.read_index("climate_index.faiss")
chunk_data = np.load("chunks.npy", allow_pickle=True)


In [ ]:
# Install Gradio
!pip install gradio

In [ ]:
import gradio as gr

def generate_response_from_query(query):
    if query.lower() in ["exit", "quit"]:
        return "Goodbye!"
    try:
        response = generate_response(query)
        return response
    except Exception as e:
        return f"An error occurred: {str(e)}"

with gr.Blocks() as climate_chatbot:
    gr.Markdown(
        "## Climate Chatbot\nAsk questions based on 'Navigating India's Climate Future'. Type 'exit' or 'quit' to end the session."
    )

    with gr.Row():
        query = gr.Textbox(
            label="Enter your question about India's climate future:",
            placeholder="Type your question here..."
        )
        output_box = gr.Textbox(label="Chatbot Response", interactive=False)

    with gr.Row():
        submit_btn = gr.Button(value="Submit", variant="primary")
        clear_btn = gr.Button(value="Clear")

    submit_btn.click(
        fn=generate_response_from_query,
        inputs=query,
        outputs=output_box
    )

    clear_btn.click(
        fn=lambda: "",
        inputs=None,
        outputs=[query, output_box]
    )

climate_chatbot.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://21ea7419516fc21a6c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
